In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_excel('output/feature_engineered_listings.xlsx')
data

,Tỉnh/Thành phố,Thành phố/Quận/Huyện/Thị xã,Xã/Phường/Thị trấn,Đường phố,Chi tiết,Nguồn thông tin,Tình trạng giao dịch,Thời điểm giao dịch/rao bán,Thông tin liên hệ,Giá rao bán/giao dịch,...,Kích thước chiều dài (m),Số mặt tiền tiếp giáp,Hình dạng,Độ rộng ngõ/ngách nhỏ nhất (m),Khoảng cách tới trục đường chính (m),Mục đích sử dụng đất,Yếu tố khác,Tọa độ (vĩ độ),Tọa độ (kinh độ),Hình ảnh của bài đăng
0,Tỉnh Khánh Hòa,Thành phố Nha Trang,Phường Phương Sơn,Đường Hải Đức,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,2800000000,...,9.00,1,Nở hậu,3.0,NaN,Đất ở,"Nhà gồm 1 trệt + 1 lửng + 1 lầu, thiết kế 4 ph...",12.252321,109.178597,"[""https://file4.batdongsan.com.vn/crop/600x315..."
1,Thành phố Hà Nội,Quận Nam Từ Liêm,Phường Tây Mỗ,Đường Tây Mỗ,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,5700000000,...,13.00,1,Chữ nhật,2.2,NaN,Đất ở,Nhà Diện tích 52m rộng rãi thoáng mát ở được g...,21.004792,105.749786,"[""https://file4.batdongsan.com.vn/crop/600x315..."
2,Thành phố Hồ Chí Minh,Quận Phú Nhuận,Phường 5,Đường Thích Quảng Đức,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,10500000000,...,34.00,1,Chữ nhật,5.0,NaN,Đất ở,| HẺM Ô TÔ 7 CHỖ - 64M2 - ĐÃ XÂY FULL KHÔNG Q...,10.805619,106.682205,"[""https://file4.batdongsan.com.vn/crop/600x315..."
3,Thành phố Hà Nội,Quận Cầu Giấy,Phường Quan Hoa,Đường Nguyễn Khánh Toàn,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,02/07/2025,NaN,18200000000,...,9.06,1,Chữ nhật,3.5,NaN,Đất ở,"Bán nhà Phố Nguyễn Khánh Toàn, Cầu Giấy, khu p...",21.037317,105.800733,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4,Thành phố Hồ Chí Minh,Quận Tân Bình,Phường 13,Đường Hoàng Hoa Thám,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,19200000000,...,11.40,1,Chữ nhật,8.0,NaN,Đất ở,"Toà nhà hẻm 8m Hoàng Hoa Thám, P.13 Tân Bình, ...",10.807920,106.688707,"[""https://file4.batdongsan.com.vn/crop/600x315..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16391,Thành phố Hà Nội,Quận Ba Đình,Phường Ngọc Hà,Đường Ngọc Hà,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,30/06/2025,NaN,29000000000,...,90.00,1,Chữ nhật,5.0,10.0,Đất ở,"Nhà nằm vị trí đắc địa, gần Lăng Bác diện tích...",21.037495,105.830136,"[""https://file4.batdongsan.com.vn/crop/600x315..."
16392,Thành phố Hà Nội,Quận Long Biên,Phường Sài Đồng,NaN,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-phuong...,Rao bán,02/07/2025,NaN,7100000000,...,5.00,2,Chữ nhật,0.0,NaN,Đất ở,Chính chủ bán nhà phố Sài Đồng - lô góc siêu đ...,21.033965,105.916267,"[""https://file4.batdongsan.com.vn/crop/600x315..."
16393,Thành phố Hồ Chí Minh,Quận Gò Vấp,Phường 16,NaN,Mặt phố,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,05/07/2025,NaN,8700000000,...,20.00,1,Chữ nhật,4.0,0.0,Đất ở,"Nhà riêng 4 tầng tại Đường Số 9, Phường 16, Gò...",10.844344,106.668257,"[""https://file4.batdongsan.com.vn/crop/600x315..."
16394,Thành phố Hồ Chí Minh,Quận 10,Phường 12,Đường Hòa Hưng,91,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,05/07/2025,NaN,81000000000,...,30.00,1,Chữ nhật,NaN,NaN,Đất ở,Bán nhà hẻm nhựa 91 Hòa Hưng P12 Q10. | - Vị t...,10.778993,106.673454,"[""https://file4.batdongsan.com.vn/crop/600x315..."


In [3]:
#data[data['Xã/Phường/Thị trấn'].isna()] == 197 rows
mask = data[data['Đường phố'].isna()]['Nguồn thông tin']
#data[(data['Xã/Phường/Thị trấn'].isna()) & (data['Đường phố'].isna())] == 29 rows
#data[data['Diện tích đất (m2)'] < data['Kích thước chiều dài (m)']]
data[data['Độ rộng ngõ/ngách nhỏ nhất (m)'].isna()].shape

(6337, 30)

In [4]:
data_raw = pd.read_csv('output/listing_details.csv')
index = []
for i in mask.to_list():
    index.append(data_raw[data_raw['url'] == i].index[0])
lack_streets = data_raw.iloc[index]

In [5]:
yo = data_raw[data_raw['description'].isna() == False]
yo

,url,title,latitude,longitude,short_address,address_parts,main_info,description,other_info,image_urls
0,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Bán nhà 3 tầng hẻm Hải Đức, P.Phương Sơn, TP. ...",12.252321,109.178597,"Đường Hải Đức, Phường Phương Sơn, Nha Trang, K...","[""Bán"", ""Khánh Hòa"", ""Nha Trang"", ""Nhà riêng t...","[{""title"": ""Mức giá"", ""value"": ""2,8 tỷ"", ""ext""...","Nhà gồm 1 trệt + 1 lửng + 1 lầu, thiết kế 4 ph...","{""Mức giá"": ""2,8 tỷ"", ""Diện tích"": ""48 m²"", ""S...","[""https://file4.batdongsan.com.vn/crop/600x315..."
1,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"NHÀ DÂN XÂY, PHÂN LÔ, CẦN BÁN GẤP GIÁ RẺ",21.004792,105.749786,"đường Tây Mỗ, Phường Tây Mỗ, Nam Từ Liêm, Hà Nội","[""Bán"", ""Hà Nội"", ""Nam Từ Liêm"", ""Nhà riêng tạ...","[{""title"": ""Mức giá"", ""value"": ""5,7 tỷ"", ""ext""...",Nhà Diện tích 52m rộng rãi thoáng mát ở được g...,"{""Mức giá"": ""5,7 tỷ"", ""Diện tích"": ""52 m²"", ""S...","[""https://file4.batdongsan.com.vn/crop/600x315..."
2,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"THÍCH QUẢNG ĐỨC,PHÚ NHUẬN, 5 TẦNG 9 PHÒNG NGỦ,...",10.805619,106.682205,"Đường Thích Quảng Đức, Phường 5, Phú Nhuận, Hồ...","[""Bán"", ""Hồ Chí Minh"", ""Phú Nhuận"", ""Nhà riêng...","[{""title"": ""Mức giá"", ""value"": ""10,5 tỷ"", ""ext...",SIÊU PHẨM THÍCH QUẢNG ĐỨC - PHÚ NHUẬN - 5 TẦNG...,"{""Mức giá"": ""10,5 tỷ"", ""Diện tích"": ""64 m²"", ""...","[""https://file4.batdongsan.com.vn/crop/600x315..."
3,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Bán nhà 82m2 18.2 tỷ, phân lô, đường Nguyễn Kh...",21.037317,105.800733,"Đường Nguyễn Khánh Toàn, Phường Quan Hoa, Cầu ...","[""Bán"", ""Hà Nội"", ""Cầu Giấy"", ""Nhà riêng tại đ...","[{""title"": ""Mức giá"", ""value"": ""18,2 tỷ"", ""ext...","Bán nhà Phố Nguyễn Khánh Toàn, Cầu Giấy, khu p...","{""Mức giá"": ""18,2 tỷ"", ""Diện tích"": ""82 m²"", ""...","[""https://file4.batdongsan.com.vn/crop/600x315..."
4,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Bán nhà riêng - giáp khu Tên Lửa - P. Tân Tạo,...",10.759390,106.599232,"302/, Đường Lê Đình Cẩn, Phường Tân Tạo, Bình ...","[""Bán"", ""Hồ Chí Minh"", ""Bình Tân"", ""Nhà riêng ...","[{""title"": ""Mức giá"", ""value"": ""Thỏa thuận"", ""...",Diện tích 62m² - hẻm 6m thông - ngang 5.13 dài...,"{""Mức giá"": ""Thỏa thuận"", ""Diện tích"": ""62 m²""...","[""https://file4.batdongsan.com.vn/crop/600x315..."
...,...,...,...,...,...,...,...,...,...,...
19964,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Bán Nhà Riêng 6PN, 4WC, 80m2, Đường Số 9, Phườ...",10.844344,106.668257,"Đường Số 9, Phường 16, Gò Vấp, Hồ Chí Minh","[""Bán"", ""Hồ Chí Minh"", ""Gò Vấp"", ""Nhà riêng tạ...","[{""title"": ""Mức giá"", ""value"": ""8,7 tỷ"", ""ext""...","Nhà riêng 4 tầng tại Đường Số 9, Phường 16, Gò...","{""Mức giá"": ""8,7 tỷ"", ""Diện tích"": ""80 m²"", ""S...","[""https://file4.batdongsan.com.vn/crop/600x315..."
19965,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Bán nhà HXH 91 Hòa Hưng P12 Q10 (15X30m Cn: 45...,10.778993,106.673454,"91, Đường Hòa Hưng, Phường 12, Quận 10, Hồ Chí...","[""Bán"", ""Hồ Chí Minh"", ""Quận 10"", ""Nhà riêng t...","[{""title"": ""Mức giá"", ""value"": ""81 tỷ"", ""ext"":...",Bán nhà hẻm nhựa 91 Hòa Hưng P12 Q10.\n- Vị tr...,"{""Mức giá"": ""81 tỷ"", ""Diện tích"": ""450 m²"", ""S...","[""https://file4.batdongsan.com.vn/crop/600x315..."
19966,https://batdongsan.com.vn/ban-nha-rieng-phuong...,Mặt tiền nội bộ phường Tân Quý 5.4x19.4m 1 lửn...,10.793344,106.624982,"Phường Tân Quý, Tân Phú, Hồ Chí Minh","[""Bán"", ""Hồ Chí Minh"", ""Tân Phú"", ""Nhà riêng t...","[{""title"": ""Mức giá"", ""value"": ""9,9 tỷ"", ""ext""...",Giá 9.9 tỷ. Nắm chủ.\nBán nhà mặt tiền nội bộ ...,"{""Mức giá"": ""9,9 tỷ"", ""Diện tích"": ""105 m²"", ""...","[""https://file4.batdongsan.com.vn/crop/600x315..."
19967,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Cần bán nhà phố liền kề Đường Ngô Chí quốc,10.886863,106.723706,"201, Đường Ngô Chí Quốc, Phường Bình Chiểu, Th...","[""Bá

In [6]:
pd.set_option('display.max_colwidth', 3)
lack_streets[['short_address', 'address_parts', 'description']]

,short_address,address_parts,description
9,"Phường Phú Hữu, Quận 9, Hồ Chí Minh","[""Bán"", ""Hồ Chí Minh"", ""Quận 9"", ""Nhà riêng tại phường Phú Hữu""]","Chủ giảm 200tr thiện chí.\nPhú Hữu TP Thủ Đức (Q9 cũ).\nChỉ còn 4,6 tỷ còn thương lượng cho khách thiện chí!\n\nKết cấu: 1 trệt 2 lầu nhà mới đẹp, thiết kế hiện đại.\nHạ tầng hoàn chỉnh khu phân lô đồng bộ, đường rộng ô tô quay đầu thoải mái.\nPháp lý chuẩn chỉnh sổ riêng, công chứng ngay.\n\nTiềm năng sinh lời cực cao nhờ vị trí nằm gần tuyến đường liên cảng Cát Lái Phú Hữu sắp hoàn thiện trục đường chiến lược dành riêng cho xe container kết nối các khu logistics lớn.\n\nThích hợp mua ở, mở văn phòng, cho thuê hoặc đầu tư chờ tăng giá.\n\nLiên hệ ngay: Mr. Luân chuyên gia bất động sản khu Đông.\nCall/Zalo:\n0909 207 ***\n.\nĐi xem nhà thực tế để cảm nhận giá trị!"
20,"Phường Đại Mỗ, Nam Từ Liêm, Hà Nội","[""Bán"", ""Hà Nội"", ""Nam Từ Liêm"", ""Nhà riêng tại phường Đại Mỗ""]","Nhà cao cửa rộng ô tô đỗ cửa 5 tầng thang máy.\nMô tả:\n- Vị trí: Trung tâm Tây Mỗ, gần Vin tiện tích đầy đủ.\n- Công viên vườn nhật, trung tâm thương mại, trường học, bệnh viện Vinmec... Chỉ cách 0.5 km.\n- Xây dựng: Khung cột chắc chắn, tháng may chạy vèo vèo.\n- Thiết kế: 5 tầng.\n+ Tầng 1: Khách bếp wc.\n+ Tầng 2: 2p ngủ wc.\n+ Tầng 3,4: 2 ngủ 2 Wc.\n+ Tầng 5: Phòng thờ, khu giặt phơi đồ.\n- Pháp lý: Sổ đỏ đất ở đô thị, chủ cất két, giao dịch nhanh."
28,"Phường Hoàng Văn Thụ, Hoàng Mai, Hà Nội","[""Bán"", ""Hà Nội"", ""Hoàng Mai"", ""Nhà riêng tại phường Hoàng Văn Thụ""]",NaN
46,"Đường Liên khu 10-11, Phường Bình Trị Đông, Bình Tân, Hồ Chí Minh","[""Bán"", ""Hồ Chí Minh"", ""Bình Tân"", ""Nhà riêng tại đường Liên khu 10-11""]","Mô tả:\n- Nhà C4 với 6 phòng trọ thu nhập 12 triệu/tháng.\n- Kích thước 6*18m.\n- Đường 5m khu dân trí, sát mặt tiền.\n- Vị trí gần Tân Hòa Đông, Phan Anh, Tô Hiệu.\n- Giáp Tân Phú gần Quận 6.\n- Sổ hồng chính chủ từ xưa, cất Két, công chứng ngay.\nLH Minh\n0909 489 ***\n."
50,"Đường 70, Phường Đại Mỗ, Nam Từ Liêm, Hà Nội","[""Bán"", ""Hà Nội"", ""Nam Từ Liêm"", ""Nhà riêng tại đường 70""]","Full nội thất - 6 tầng thang máy, xây đơn lập - sổ vuông bánh chưng, ngõ rộng 3m, khu vực hot hit nhất đại mỗ - đón đầu đường Lê Quang Đạo kéo dài - tương lai sáng ngời.\n- Nhà xây đơn lập.\n- Thiết kế 6 tầng, thang máy.\n+ Tầng 1: Để xe, sinh hoạt chung.\n+ Tầng 2,3,4 mỗi tầng 1 ngủ khép kín.\n+ Tầng 5: Phòng khách rất rộng.\n+ Tầng 6: Bếp. Khu giặt phơi, vườn thượng uyển, khu vực tiểu cảnh cafe, trà ngắm TTTM Aeon Hà Đông.\n- Nhà hoàn thiện full nội thất các tầng như hình. Tủ bếp, tủ quần áo, giường, bàn ăn, sofa...\n- Ngõ to, rộng 3m có dư. Khu vực đang hot hit số 1 đất Đại Mỗ.\n- Cách đường Lê Quang Đạo kéo dài chỉ 300m, cách UBND Đại Mỗ chỉ 100m. Tiện ích ko thiếu thứ gì.\n- Với giá tiền 7.5 tỷ mua nhà Mỹ Đình phải vào trong ngõ sâu, 4 tầng thang bộ, sang Đại Mỗ có ngay nhà 6 tầng thang máy, ôtô đỗ cổng ở sướng vô cùng.\n- Có bãi gửi xe rộng tại nhà văn hóa cách nhà 30m.\n- Sổ cất két, giao dịch ngay."
...,...,...,...
19954,"Phường Phú Đô, Nam Từ Liêm, Hà Nội","[""Bán"", ""Hà Nội"", ""Nam Từ Liêm"", ""Nhà riêng tại phường Phú Đô""]","Chỉ 7.3 tỷ! Bán nhà 41m2x4t Phú Đô, Nam Từ Liêm, ngõ nông rộng 2.5m, gần ô tô, sát phố, gần trường học.\n\nVị trí đẹp, ngõ thẳng, ba gác vào nhà, 40m ra ngõ thông ô tô, đi bộ vài bước ra trường tiểu học, trung học Phú Đô, chợ... Sát mặt phố Lê Quang Đạo thông ra Châu Văn Liêm, Đại Lộ Thăng Long, Mễ Trì, Phạm Hùng. Gần sát sân Mỹ Đình, bệnh viện Hồng Ngọc, bộ ngoại giao... Các KĐT Mễ Trì, Mỹ Đình Sông Đà... Quá tiện ích.\n\nSổ đẹp A4 sẵn sàng giao dịch ngay.\n\nLH E Tráng nhà đẹp\n0973 136 ***\nxem nhà ngay. Làm việc chính chủ."
19955,"Phường Bưởi, Tây Hồ, Hà Nội","[""Bán"", ""Hà Nội"", ""Tây Hồ"", ""Nhà riêng tại phường Bưởi""]","- Diện tích 43m² xây 5 tầng chắc chắn 4PN\n- Ngõ thẳng, nông gần mặt phố 30m ra ô tô\n- Sổ đỏ vuông vắn pháp lý minh bạch\n- Hợp khách mua ở hoặc đầu tư cho thuê căn hộ\n- Call/zalo\n0902 165 ***\nđể nhận video xem nhà nếu c

In [ ]:
yo['split_des'] = yo['description'].str.split('\n')
yo['split_des'] = yo['split_des'].apply(lambda x: [i.split() for i in x])
yo['split_des']

C:\Users\Admin\AppData\Local\Temp\ipykernel_3240\1694072946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yo['split_des'] = yo['description'].str.split('\n')
C:\Users\Admin\AppData\Local\Temp\ipykernel_3240\1694072946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yo['split_des'] = yo['split_des'].apply(lambda x: [i.split() for i in x])


0        [[Nhà, gồm, 1, trệt, +, 1, lửng, +, 1, lầu,, thiết, kế, 4, phòng, ngủ, và, 3, toilet,, sân, để, xe, rộng.], [Vị, trí, nhà, tiếp, giáp, hẻm, rộng, 3m,, cách, đường, 23/10, chỉ, 70m., Khu, vực, dân, cư, đông, đúc,, dân, trí, cao.], [Diện, tích:, 48m²., Ngang, 3,5, x, dài, 9m, x, nở, hậu, 8m.], [Sổ, hồng, chính, chủ., Hướng, Tây, Nam.], [Giá, bán:, 2, tỷ, 800, triệu.], [Vui, lòng, liên, hệ, anh, Tám:], [0901, 904, ***], [.]]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [9]:
def check_san(row):
    for i in row:
        if 'sàn' in i:
            return True
        return False

filtered_df = yo[yo['split_des'].apply(check_san)]
filtered_df.shape

(38, 11)

In [ ]:
filtered_df['description'].iloc[35:]
# 6, 652, 2839, 5817, 6745, 7077, 7324, 7347, 8241, 8527, 9919, 10307, 12236, 13415, 13454, 15375, 15996, 16761, 16793, 16870, 18053, 18147 (22 gia tri)

18738    Nhà Đẹp Tây Mỗ # Lô góc thoáng sáng vĩnh viễn, nhà mặt tiền rộng tận 8 m , rất nhiều cửa sổ , rất thoáng mát và sinh khí tốt + Nhà xây 5 tầng đủ , sàn gỗ và Thạch cao FUN cả nhà , hoàn thiện tỷ mỷ , trang thiết bị đầy đủ , chỉ việc xách valy vào ở ngay...+ Pháp lý chuẩn, sổ đỏ sẵn sàng. Lh ngay em Hảo " Đất tốt "\n0904 529 ***\n.
18747    Nhà mới xây 6 tầng, thang máy xịn, chữa cháy tự động từng phòng, lối thoát hiểm riêng an toàn, nhà 2 mặt ngõ trước sau, phòng nào cũng có ban công thoáng, sÁng, đẹp. Thiết kế sàn 2 phòng,\nKHEP KÍN FULL NỘI THẤT mỗi phòng rộng 4 người ở giường tầng. Cho thuê giá 6 triệu một phòng.                                                  
19148    Thiết kế thông sàn 5 phòng ngủ rộng, thiết bị vệ sinh đầy đủ bàn giao luôn.\nTiện ích xung quanh gần chợ trường học lớn như Công Đoàn Thủy Lợi, cao đẳng Y, bệnh viện Xanh Pôn, Công Viên Thống Nhất, vài phút lên Phố Cổ.\n\nSổ đẹp nở hậu sang tên ngay.                                                           

- (done)diện tích sàn / DT sàn / Diện tích sàn: 37m² / tổng diện tích sàn lên đến 340,4m² (index 83), DT: 5,5 x 12, sàn 91 m² / Diện Tích Sàn XD 124.8m2
- (done) 55m * 5T
- DT xây dựng 4x20m trệt 1 lầu thành 5 căn riêng biệt
- diện tích xây dựng 56m² (4m x 14m)

In [176]:
yo.columns

Index(['url', 'title', 'latitude', 'longitude', 'short_address',
       'address_parts', 'main_info', 'description', 'other_info', 'image_urls',
       'split_des'],
      dtype='object')

In [ ]:
def extract_built_area(row):
    des = row['description'].lower()
    
    first_pattern = re.search(pattern = r"(?:diện\s+tích\s+|dt\s+)?sàn(?:\s+\S+){0,5}?\s*:?\s*(\d+[.,]?\d*)\s*(?:m2|m²|m)\b", string = des)
    if first_pattern:
        result = first_pattern.group(1)
        if ',' in result:
            result = result.replace(',', '.')
        return float(result)
    else:
        second_pattern = re.search(pattern = r'(\d+[.,]?\d*)\s*m\s*[*|x]\s*(\d+)\s*[T|t]', string=des)
        if second_pattern:
            area = second_pattern.group(1)
            floor = second_pattern.group(2)
            if ',' in area:
                area = area.replace(',', '.')
            
            return float(area) * float(floor)
        else:
            return None



0        None
1        None
2        None
3        None
4        None
         ... 
19964    None
19965    None
19966    None
19967    None
19968    None
Length: 19957, dtype: object

In [171]:
# Diện tích sàn / dt sàn / words linh tinh nữa
examples = [
    "Diện tích sàn tầng trệt 100m2",  # ✅ 2 words in between
    "Diện tích sàn tầng trệt lầu và sân 150m²",  # ✅ 5 words
    "Diện tích sàn lầu 1 và sân thượng có mái che 200m2",  # ❌ 6 words → no match
    'DT sàn : 67m2',
    'tổng Diện tích sàn lên đến 340,4m²',
    'sàn 95 m2',
    'Diện Tích Sàn XD 124.8m2'
]

# for s in examples:
#     m = re.search(r"\s*(Diện tích|DT)? sàn(?:\s+\S+){0,6}\s?:?\s+(\d+[,.]?\d*)\s*(m2|m²|m)\b", s)
#     print(m.group(2) if m else "No match")

# Dù sao sau này cũng lower trước nên không cần phải re.IGNORECASE
pattern = re.compile(
    r"(?:diện\s+tích\s+|dt\s+)?sàn(?:\s+\S+){0,5}?\s*:?\s*(\d+[.,]?\d*)\s*(?:m2|m²|m)\b",
    re.IGNORECASE
)

for s in examples:
    m = pattern.search(s)
    if m:
        result = m.group(1)
        if ',' in result:
            result = result.replace(',', '.')
        print(float(result))
    else:
        print('No match')

100.0
150.0
No match
67.0
340.4
95.0
124.8


In [188]:
# 55m * số tầng
examples = ['45,8 m * 5 t',
            '45.8m x 5t',
            '239,1m*1t', '24 m * 5 tầnfg']
import re
#yessir = yo[yo['description'].str.contains(r'(\d+)\s*m\s*[*|x]\s*\d+\s*[T|t]')].drop(['split_des'], axis=1)
for i in examples:
    m = re.search(pattern = r'(\d+[.,]?\d*)\s*m\s*[*|x]\s*(\d+)\s*[T|t]', string=i)
    print(m.group(0))
    print(m.group(1))
    print(m.group(2))

45,8 m * 5 t
45,8
5
45.8m x 5t
45.8
5
239,1m*1t
239,1
1
24 m * 5 t
24
5


In [2]:
import pandas as pd

cleaned = pd.read_csv('output/listing_details_cleaned.csv')
cleaned.columns

Index(['Tỉnh/Thành phố', 'Thành phố/Quận/Huyện/Thị xã', 'Xã/Phường/Thị trấn',
       'Đường phố', 'Chi tiết', 'Nguồn thông tin', 'Tình trạng giao dịch',
       'Thời điểm giao dịch/rao bán', 'Thông tin liên hệ',
       'Giá rao bán/giao dịch', 'Loại đơn giá (đ/m2 hoặc đ/m ngang)',
       'Số tầng công trình', 'Tổng diện tích sàn', 'Đơn giá xây dựng',
       'Năm xây dựng', 'Chất lượng còn lại', 'Diện tích đất (m2)',
       'Kích thước mặt tiền (m)', 'Kích thước chiều dài (m)',
       'Số mặt tiền tiếp giáp', 'Hình dạng', 'Độ rộng ngõ/ngách nhỏ nhất (m)',
       'Khoảng cách tới trục đường chính (m)', 'Mục đích sử dụng đất',
       'Yếu tố khác', 'Tọa độ (vĩ độ)', 'Tọa độ (kinh độ)',
       'Hình ảnh của bài đăng', 'description'],
      dtype='object')

In [5]:
cleaned[cleaned['Tổng diện tích sàn'] <= 0]

,Tỉnh/Thành phố,Thành phố/Quận/Huyện/Thị xã,Xã/Phường/Thị trấn,Đường phố,Chi tiết,Nguồn thông tin,Tình trạng giao dịch,Thời điểm giao dịch/rao bán,Thông tin liên hệ,Giá rao bán/giao dịch,...,Số mặt tiền tiếp giáp,Hình dạng,Độ rộng ngõ/ngách nhỏ nhất (m),Khoảng cách tới trục đường chính (m),Mục đích sử dụng đất,Yếu tố khác,Tọa độ (vĩ độ),Tọa độ (kinh độ),Hình ảnh của bài đăng,description
152,Thành phố Hồ Chí Minh,Quận Bình Thạnh,Phường 11,Đường Phan Văn Trị,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,04/07/2025,NaN,1.080000e+10,...,1,Chữ nhật,8.0,95.0,Đất ở,"Bán nhà văn phòng đẹp 3 tầng 6x14m, hẻm xe tải...",10.815878,106.694750,"[""https://file4.batdongsan.com.vn/crop/600x315...","Bán nhà văn phòng đẹp 3 tầng 6x14m, hẻm xe tải..."
8056,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Phố Liễu Giai,Mặt phố,https://batdongsan.com.vn/ban-nha-rieng-pho-li...,Rao bán,04/07/2025,NaN,1.250000e+11,...,2,Chữ nhật,0.0,0.0,Đất ở,"- Siêu hiếm lô góc Liễu Giai, Ba Đình, 106m²/M...",21.033927,105.813855,"[""https://file4.batdongsan.com.vn/crop/600x315...","- Siêu hiếm lô góc Liễu Giai, Ba Đình, 106m²/M..."
10531,Thành phố Hà Nội,Quận Tây Hồ,Phường Xuân La,Đường Võ Chí Công,Mặt phố,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,30/06/2025,NaN,1.280000e+10,...,1,Chữ nhật,0.0,0.0,Đất ở,"Siêu phẩm Võ Chí Công mặt tiền khủng 6m, 6 tần...",21.059674,105.804476,"[""https://file4.batdongsan.com.vn/crop/600x315...","Siêu phẩm Võ Chí Công mặt tiền khủng 6m, 6 tần..."


In [ ]:
def fill_built(row):
    if pd.isna(row['Tổng diện tích sàn']):
        if pd.isna(row['Diện tích đất (m2)']) == False and pd.isna(row['Số tầng công trình']) == False:
            return row['Diện tích đất (m2)'] * row['Số tầng công trình']
        return None
    else:
        return row['Tổng diện tích sàn']
    
yo = cleaned.apply(fill_built, axis = 1)

Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN
We're here
Đang bị NaN

In [12]:
fe = pd.read_excel('output/feature_engineered_listings.xlsx')
fe.columns

Index(['Tỉnh/Thành phố', 'Thành phố/Quận/Huyện/Thị xã', 'Xã/Phường/Thị trấn',
       'Đường phố', 'Chi tiết', 'Nguồn thông tin', 'Tình trạng giao dịch',
       'Thời điểm giao dịch/rao bán', 'Thông tin liên hệ',
       'Giá rao bán/giao dịch', 'Giá ước tính',
       'Loại đơn giá (đ/m2 hoặc đ/m ngang)', 'Đơn giá đất',
       'Lợi thế kinh doanh', 'Số tầng công trình', 'Tổng diện tích sàn',
       'Đơn giá xây dựng', 'Năm xây dựng', 'Chất lượng còn lại',
       'Diện tích đất (m2)', 'Kích thước mặt tiền (m)',
       'Kích thước chiều dài (m)', 'Số mặt tiền tiếp giáp', 'Hình dạng',
       'Độ rộng ngõ/ngách nhỏ nhất (m)',
       'Khoảng cách tới trục đường chính (m)', 'Mục đích sử dụng đất',
       'Yếu tố khác', 'Tọa độ (vĩ độ)', 'Tọa độ (kinh độ)',
       'Hình ảnh của bài đăng', 'Số ngày tính từ lúc đăng tin'],
      dtype='object')

In [13]:
fe[['Thời điểm giao dịch/rao bán', 'Số ngày tính từ lúc đăng tin']].head()

,Thời điểm giao dịch/rao bán,Số ngày tính từ lúc đăng tin
0,2025-07-07,16
1,2025-07-07,16
2,2025-07-07,16
3,2025-07-02,21
4,2025-07-07,16


In [22]:
fe[(fe['Tổng diện tích sàn'].isna()) & (fe['Giá rao bán/giao dịch'].isna())].shape

(1489, 31)

In [15]:
import pandas as pd

new_data = pd.read_excel('output/ml_imputed_listings.xlsx')
new_data

FileNotFoundError: [Errno 2] No such file or directory: 'output/ml_imputed_listings.xlsx'

In [4]:
type(new_data['Thời điểm giao dịch/rao bán'].iloc[0])

str

In [8]:
new_data.columns

Index(['Tỉnh/Thành phố', 'Thành phố/Quận/Huyện/Thị xã', 'Xã/Phường/Thị trấn',
       'Đường phố', 'Chi tiết', 'Nguồn thông tin', 'Tình trạng giao dịch',
       'Thời điểm giao dịch/rao bán', 'Thông tin liên hệ',
       'Giá rao bán/giao dịch', 'Giá ước tính',
       'Loại đơn giá (đ/m2 hoặc đ/m ngang)', 'Đơn giá đất',
       'Lợi thế kinh doanh', 'Số tầng công trình', 'Tổng diện tích sàn',
       'Đơn giá xây dựng', 'Năm xây dựng', 'Chất lượng còn lại',
       'Diện tích đất (m2)', 'Kích thước mặt tiền (m)',
       'Kích thước chiều dài (m)', 'Số mặt tiền tiếp giáp', 'Hình dạng',
       'Độ rộng ngõ/ngách nhỏ nhất (m)',
       'Khoảng cách tới trục đường chính (m)', 'Mục đích sử dụng đất',
       'Yếu tố khác', 'Tọa độ (vĩ độ)', 'Tọa độ (kinh độ)',
       'Hình ảnh của bài đăng'],
      dtype='object')

In [5]:
import pandas as pd

hello = pd.DataFrame({'Cat': ['h', 'e', 'l', 'l', 'o'], 'Num': [1, 2, 3, 4, 5]})
hello = pd.get_dummies(hello, columns=['Cat'], drop_first=False)
hello

,Num,Cat_e,Cat_h,Cat_l,Cat_o
0,1,False,True,False,False
1,2,True,False,False,False
2,3,False,False,True,False
3,4,False,False,True,False
4,5,False,False,False,True
